# Install Dependencies

In [1]:
%pip install "comet_ml>=3.38.0" torch torchvision tqdm &> /dev/null

Note: you may need to restart the kernel to use updated packages.


# Initialize Comet

In [2]:
import comet_ml
from comet_ml.integration.pytorch import watch
from comet_ml import Experiment

experiment = Experiment(
    api_key="add your comet API key",
    project_name="yout project name",
    workspace="your workspace name",
    log_git_metadata=False, log_git_patch=False,display_summary_level = 0,
)
experiment.set_name(f"your experiment name")
experiment.add_tags(["test","model_RNN", "my tag"])


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/kaggle/working' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/mortezaomidi24/ml-tools-test/2c0660d88f954315abcde0cd6c53aa39



# Import Dependencies

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable
from tqdm import tqdm

# Define Parameters

In [5]:
hyper_params = {"batch_size": 100, "num_epochs": 3, "learning_rate": 0.01}
experiment.log_parameters(hyper_params)

# Load Data

In [6]:
# MNIST Dataset
train_dataset = datasets.MNIST(
    root="./data/", train=True, transform=transforms.ToTensor(), download=True
)

test_dataset = datasets.MNIST(
    root="./data/", train=False, transform=transforms.ToTensor()
)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=hyper_params["batch_size"], shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=hyper_params["batch_size"], shuffle=False
)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:03<00:00, 2509329.70it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 333794.50it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 2685296.06it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 1844194.46it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# Define Model and Optimizer

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)

        return x


model = Net().to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params["learning_rate"])

# Train a Model

In [8]:
def train(model, optimizer, criterion, dataloader, epoch, experiment):
    model.train()
    total_loss = 0
    correct = 0
    for batch_idx, (images, labels) in enumerate(
        tqdm(dataloader, total=len(dataloader))
    ):
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)

        loss = criterion(outputs, labels)
        pred = outputs.argmax(
            dim=1, keepdim=True
        )  # get the index of the max log-probability

        loss.backward()
        optimizer.step()

        # Compute train accuracy
        batch_correct = pred.eq(labels.view_as(pred)).sum().item()
        batch_total = labels.size(0)

        total_loss += loss.item()
        correct += batch_correct

        # Log batch_accuracy to Comet; step is each batch
        experiment.log_metric("batch_accuracy", batch_correct / batch_total)

    total_loss /= len(dataloader.dataset)
    correct /= len(dataloader.dataset)

    experiment.log_metrics({"accuracy": correct, "loss": total_loss}, epoch=epoch)

In [9]:
# Train the Model
print("Running Model Training")

max_epochs = hyper_params["num_epochs"]
with experiment.train():
    watch(model)
    for epoch in range(max_epochs + 1):
        print("Epoch: {}/{}".format(epoch, max_epochs))
        train(model, optimizer, criterion, train_loader, epoch, experiment)

Running Model Training
Epoch: 0/3


100%|██████████| 600/600 [01:18<00:00,  7.60it/s]


Epoch: 1/3


100%|██████████| 600/600 [01:23<00:00,  7.23it/s]


Epoch: 2/3


100%|██████████| 600/600 [01:22<00:00,  7.26it/s]


Epoch: 3/3


100%|██████████| 600/600 [01:23<00:00,  7.18it/s]


# Evaluate Model

In [ ]:
def test(model, optimizer, criterion, dataloader, epoch, experiment):
    model.eval()

    total_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (images, labels) in tqdm(enumerate(dataloader)):
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            total_loss += criterion(output, labels).item()  # sum up batch loss
            pred = output.argmax(
                dim=1,
                keepdim=True,
            )  # get the index of the max log-probability
            correct += pred.eq(labels.view_as(pred)).sum().item()

        total_loss /= len(dataloader.dataset)
        correct /= len(dataloader.dataset)

        experiment.log_metrics({"accuracy": correct, "loss": total_loss}, epoch=epoch)

In [ ]:
# Test the Model
print("Running Model Evaluation")

with experiment.test():
    test(model, optimizer, criterion, test_loader, epoch, experiment)

# Log the Model to Comet

In [ ]:
from comet_ml.integration.pytorch import log_model

log_model(experiment, model, "Pytorch-Net-FashionMNIST")

# End Experiment 

In [ ]:
experiment.end()